In [8]:
from pytwitter import StreamApi
import json
import os
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 


In [33]:
class MyStream(StreamApi):
    def __init__(self, bearer_token):
        StreamApi.__init__(self, bearer_token=bearer_token)
        self.__file_index = 0
    
    def on_tweet(self, tweet):
        print(tweet)
        tweets_file_path = '/Users/yhua/Desktop/Real-time-Sentiment-Tracking-on-Twitter-for-Brand-Improvement-and-Trend-Recognition/pytwitter/tweets_json/{}.json'\
        .format(self.__file_index)
        with open(tweets_file_path, 'w', encoding='utf-8') as outfile:
                json.dump(tweet, outfile, ensure_ascii=False)
        while os.path.exists(tweets_file_path) and os.stat(tweets_file_path).st_size > 2**10:
            self.__file_index += 1 
            tweets_file_path = '/Users/yhua/Desktop/Real-time-Sentiment-Tracking-on-Twitter-for-Brand-Improvement-and-Trend-Recognition/pytwitter/tweets_json/{}.json'\
            .format(self.__file_index)

            

In [34]:
stream = MyStream(bearer_token=credentials.BEARER_TOKEN)
stream.sample_stream(return_json=True)

{'data': {'id': '1544781238933798914', 'text': 'RT @ALoetje: @geertwilderspvv @peterstor @GidiMarkuszower En wat heeft hij gelijk, als je het woord eng al niet meer mag zeggen. Het lijkt…'}}
{'data': {'id': '1544781238917005312', 'text': 'RT @marl1too: if you didn’t make fake youtube videos when u were younger-do you even exist??'}}
{'data': {'id': '1544781238933901312', 'text': 'RT @talitations: @tsuki696 dps envelhece sozinha no asilo e as pessoas dizem que os filhos são ingratos'}}
{'data': {'id': '1544781238921318401', 'text': 'رفتم يكم جوجه كباب و نون زيرش رو بذارم واسه گربه هاى كوچه، يه دختر ادايى پتياره هاپار هـاپار حمله كرد بهم كه چرا غذاى ادويه دار و چرب ميدى بهشون بايد غذا خشك بدى! همينجور وحشى ايد كه آدما از حيوون دوستى زده ميشن ديگه! البته از خجالتش كه درومدم بنده 🥰'}}
{'data': {'id': '1544781238908723200', 'text': '🖤\U0001f90d https://t.co/D5PzVB1vVl'}}
{'data': {'id': '1544781238921269249', 'text': 'RT @meleniaran: BAL "Hubo asesinados injustamente, en los dos bandos"\n¡N

KeyboardInterrupt: 

Initialized empty Git repository in /Users/yhua/Desktop/twitter_analysis/.git/
